# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas 
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [40]:
file_path = "../output_data/cities.csv"

# Read the CSV file into a DataFrame
city_data_df = pd.read_csv(file_path)
city_data_df.set_index('City_ID', inplace=True)
columns_to_drop = [col for col in city_data_df.columns if 'City_ID' in col or 'Unnamed' in col]
city_data_df.drop(columns=columns_to_drop, inplace=True)

# Display sample data
display(city_data_df)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...
572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564
574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565


In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    scale=0.5,  # Adjust scale as needed
    color='City',
    hover_cols=['City', 'Humidity']
)

# Display map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [96]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[(city_data_df["Humidity"] < 50) & (city_data_df["Max Temp"] > 25)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
103,siavonga,-16.5382,28.7088,31.47,32,71,1.58,ZM,1666108296
105,nova olimpia,-14.7972,-57.2881,40.11,21,93,1.65,BR,1666108147


### Step 3: Create a new DataFrame called `hotel_df`.

In [99]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(len(hotel_df.columns), 'Hotel Name', '')

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
22,guiratinga,BR,-16.3494,-53.7583,14,
29,saint-louis,SN,16.3333,-15.0000,24,
73,nalut,LY,30.3333,10.8500,27,
103,siavonga,ZM,-16.5382,28.7088,32,
105,nova olimpia,BR,-14.7972,-57.2881,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [102]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
api_key = "dc68340a87af4b0f9b3705f637f8485b"
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Assuming hotel_df has columns 'City', 'lat', and 'lon'
# Add a new column for the hotel name
hotel_df['Hotel Name'] = None

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
sao miguel do araguaia - nearest hotel: Hotel São Miguel
aripuana - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
ouallam - nearest hotel: No hotel found
shakawe - nearest hotel: EdenWood guesthouse
upington - nearest hotel: Die Eiland Holiday Ressort
gwanda - nearest hotel: No hotel found
cabra - nearest hotel: Pensión Guerrero
prieska - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
sakakah - nearest hotel: فندق النزل
camopi - nearest hotel: Chez Lucia
mandera - nearest hotel: Kornesh Hotel and Lodging
numan - nearest hotel: No hotel found
beitbridge - nearest hotel: Beitbridge Inn & Casino
adrar - nearest hotel: No hotel found
buraydah - nearest hotel: دنا للوحدات السكنية
bogande - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
22,guiratinga,BR,-16.3494,-53.7583,14,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
73,nalut,LY,30.3333,10.8500,27,No hotel found
103,siavonga,ZM,-16.5382,28.7088,32,Lake Safari Lodge Annex
105,nova olimpia,BR,-14.7972,-57.2881,21,Olímpia Hotel
162,sao miguel do araguaia,BR,-13.2750,-50.1628,13,Hotel São Miguel
166,aripuana,BR,-9.1667,-60.6333,45,No hotel found
170,sur,OM,22.5667,59.5289,40,Sur Hotel
175,ouallam,NE,14.3188,2.0866,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [91]:
%%capture --no-display

merged_df = pd.merge(city_data_df, hotel_df[['City', 'Lat', 'Lng', 'Hotel Name', 'Country']], on=['City', 'Lat', 'Lng'], how='left')

# Configure the map plot
map_plot = merged_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    scale=0.5,  # Adjust scale as needed
    color='City',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country']
)

# Display map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)